In [2]:
import sys, os
from os import path
sys.path.append('/root/bdrisk/risk_project')
sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
from risk_models import *
from risk_models import _name_BD_RISK_RESULT_PRODUCTION_PD1, _name_BD_RISK_RESULT_PRODUCTION_PD2, _name_BD_RISK_RESULT_PRODUCTION_PD3, _name_BD_RISK_RESULT_PRODUCTION_PD4


class ScorePd(object):
    def __init__(self, child_task_id, org_code, params):
        self.child_task_id = child_task_id
        Risk_logger(child_task_id = self.child_task_id, exec_status=None)

        # 参数读取
        self.org_code = org_code
        self.weights = json.loads(params)['weights']

    def score_production(self):
        # 读取企业库存模块的结果表数据
        PD1_RESULT = Read_Oracle().read_oracle(sql= """ select * from {} where org_code = '{}' and iscurrent = 1 """.format(_name_BD_RISK_RESULT_PRODUCTION_PD1, self.org_code), database = 'dbods')
        PD2_RESULT = Read_Oracle().read_oracle(sql= """ select * from {} where org_code = '{}' and iscurrent = 1 """.format(_name_BD_RISK_RESULT_PRODUCTION_PD2, self.org_code), database = 'dbods')
        PD3_RESULT = Read_Oracle().read_oracle(sql= """ select * from {} where org_code = '{}' and iscurrent = 1 """.format(_name_BD_RISK_RESULT_PRODUCTION_PD3, self.org_code), database = 'dbods')
        PD4_RESULT = Read_Oracle().read_oracle(sql= """ select * from {} where org_code = '{}' and iscurrent = 1 """.format(_name_BD_RISK_RESULT_PRODUCTION_PD4, self.org_code), database = 'dbods')
        RESULT_LIST = [PD1_RESULT, PD2_RESULT, PD3_RESULT, PD4_RESULT]
        
        # 计算各个子模块分数（没有数据的初始化为0分）
        for i in range(4):
            if RESULT_LIST[i] is None:
                RESULT_LIST[i] = 0
            else:
                RESULT_LIST[i] = 100 + RESULT_LIST[i]['SCORE'].sum()
        
        # 设置子模块权重(通过json参数获取)
        RATIO = self.weights
        PRODUCTION_SCORE = 0
        
        # 对所有子模型分数加权计算总分
        for i in range(4):
            PRODUCTION_SCORE = PRODUCTION_SCORE + RESULT_LIST[i] * RATIO[i]
        
        # 整理分数表并写入数据库
        PRODUCTION_RESULT = pd.DataFrame(data=[[0, self.org_code, 'PRODUCTION', PRODUCTION_SCORE]], columns=['ID', 'ORG_CODE', 'MODEL_CODE', 'SCORE'])
        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        PRODUCTION_RESULT['CHECK_TIME'] = datetime.datetime.strptime(now, "%Y-%m-%d %H:%M:%S")
        Write_Oracle().write_oracle('BD_RISK_CORP_SCORE_DISPLAY',PRODUCTION_RESULT, org_code = self.org_code, alarm = ['PRODUCTION',''])

    def run_score_production(self):
        try:
            self.score_production()
            exec_status = 1
        except:
            logger.exception('model execution error')
            exec_status = 0
        finally:
            Risk_logger(child_task_id = self.child_task_id, exec_status=exec_status).write_log()


if __name__ == '__main__':
    # 读取传入的 child_task_id，仅适用于py调用,先备注掉
    # child_task_id = sys.argv[1]
    child_task_id = '0003_0002'
    org_code, params, base_time = read_log_table(child_task_id)
    ScorePd(child_task_id, org_code, params).run_score_production()



2021-06-07 17:28:58.502 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 7.547s
2021-06-07 17:29:13.011 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 14.50s
2021-06-07 17:29:14.499 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 1.484s
2021-06-07 17:29:14.729 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.226s
2021-06-07 17:29:14.858 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.089s
2021-06-07 17:29:15.026 | INFO     | risk_models.config.write_config.write_func:write_oracle:179 - Processing... Writing 1 rows into database
2021-06-07 17:29:15.052 | INFO     | risk_models.config.write_config.write_func:write_oracle:186 - Inse